In [1]:
import os
import sys
import json
import pandas as pd
import numpy as np
import torch
from typing import Literal, TypedDict

In [2]:
sys.path.append("..")
os.environ["USER_PATH"] = "../userdata/"

In [3]:
from system.namespace.store import get_namespace

In [4]:
namespace = get_namespace("train_db")

In [5]:
from system.embedding.store import get_embed_store
from misc.lru import LRU

In [6]:
embeds = get_embed_store(namespace)

In [7]:
role_c = "child"
role_p = "parent"

In [8]:
total_c = embeds.get_embedding_count(role_c)
total_c

689414

In [9]:
total_p = embeds.get_embedding_count(role_p)
total_p

689415

In [10]:
%%time

# x_0 = np.vstack([
#     embed[1].ravel().detach().numpy()
#     for embed in embeds.get_all_embeddings(role_c, progress_bar=True)
# ])
# y_0 = x_0.shape[0]
# x_1 = np.vstack([
#     embed[1].ravel().detach().numpy()
#     for embed in embeds.get_all_embeddings(role_p, progress_bar=True)
# ])
# y_1 = x_1.shape[0]
# y = [
#     "tab:blue" for _ in range(y_0)
# ] + [
#     "tab:orange" for _ in range(y_1)
# ]
# x = np.vstack([x_0, x_1])
# x.shape

CPU times: user 1 µs, sys: 1e+03 ns, total: 2 µs
Wall time: 2.15 µs


In [11]:
# x[:2, :]

In [12]:
def sigmoid(x):  
    return np.exp(-np.logaddexp(0, -x))

def distance(a, b):
    return np.log1p(np.exp(-np.dot(a, b)))
    # return 1.0 - sigmoid(np.dot(a, b))

In [13]:
total = total_c + total_p
# dists = np.zeros((total, total))
total * total / 1024 / 1024 / 1024

1770.6019908571616

In [14]:
ea = torch.DoubleTensor([-0.04514108598232269,0.04793350398540497,0.37372925877571106,-0.18735022842884064,0.6874262094497681,0.1700430065393448,0.5189236402511597,0.2124776989221573,-0.2540300488471985,-0.2601666748523712,-0.08945835381746292,0.028923355042934418,-0.6344109177589417,0.38846924901008606,-0.32380345463752747,0.8652123808860779,1.0186213254928589,-0.022720133885741234,-0.30368518829345703,-0.22736042737960815,0.21934981644153595,-0.03484392538666725,0.31107616424560547,0.7124900817871094,0.5669865608215332,0.05556293949484825,0.5369626879692078,0.03702433034777641,-0.21854043006896973,0.24118757247924805,-0.03484503552317619,0.00022505043307319283,0.21472881734371185,-0.22760659456253052,-0.340795636177063,0.22289752960205078,-0.424188494682312,-0.20882917940616608,-0.647316038608551,-0.05615193024277687,-0.8380041718482971,-0.5820614695549011,-0.5032729506492615,0.19232502579689026,0.22814077138900757,0.13140007853507996,-0.3146217167377472,0.7981080412864685,-0.15888327360153198,-0.09261488169431686,-0.6413535475730896,0.039709851145744324,0.1885634958744049,0.0937650129199028,0.20454806089401245,0.43915021419525146,-0.15323220193386078,-0.6929309964179993,-0.46361252665519714,-0.33070674538612366,-0.05911180004477501,0.48092833161354065,0.5252285003662109,-1.027598261833191,0.1342015117406845,-0.034821767359972,0.22844718396663666,0.7029098868370056,-1.128738522529602,-0.8171422481536865,-0.16561397910118103,-0.048531100153923035,-0.3585357367992401,-0.504396915435791,0.127089723944664,0.11059322953224182,-0.40066736936569214,0.7528361678123474,0.3588479161262512,0.07626207172870636,-0.37918612360954285,-0.013629574328660965,0.12373864650726318,0.7062016129493713,-0.0003270865126978606,-0.3311047852039337,-0.11209771782159805,0.14426659047603607,0.009177396073937416,0.42218878865242004,-0.8451879024505615,-0.7361544966697693,0.10809388756752014,0.16072149574756622,0.44531241059303284,0.011211886070668697,0.527932345867157,-0.0708528682589531,-0.18570737540721893,-0.006503901910036802,0.07573378831148148,-0.48588797450065613,0.2737022042274475,0.014893580228090286,-0.2687094509601593,-0.18467006087303162,-0.07875026017427444,-0.8259641528129578,-0.030962975695729256,0.6439911127090454,-0.054169535636901855,-0.2279801219701767,-0.16651198267936707,-0.6958741545677185,-0.554573118686676,0.06843245774507523,0.19185376167297363,-0.6216809749603271,-0.007143780589103699,0.012417715042829514,0.6527696847915649,-0.030104197561740875,0.10173490643501282,1.3972711563110352,-0.1611461192369461,-0.1345949023962021,-0.1839502900838852,0.7162511348724365,-1.0126982927322388,-0.15339642763137817,0.25200703740119934,0.4142036736011505,0.02282005548477173,0.004447871819138527,-0.45930635929107666,-0.5861625075340271,0.38590461015701294,-0.0758097693324089,-0.2175416797399521,0.23816563189029694,0.12176530063152313,-0.3622192144393921,0.026671428233385086,0.19927097856998444,0.0990205928683281,0.04916542395949364,-0.02591065689921379,-0.6908756494522095,-0.06604376435279846,-0.08623619377613068,-0.002380779478698969,-0.05074912682175636,-0.17951440811157227,0.023849565535783768,-0.2637968361377716,0.1027757003903389,-0.04435760900378227,0.3452152609825134,-0.07399404048919678,0.033584900200366974,0.4278450012207031,-0.030071508139371872,0.30976641178131104,-0.269827276468277,0.5861563682556152,0.13085220754146576,-0.015281387604773045,0.9301788806915283,-0.1777561902999878,0.4558124840259552,-0.08555159717798233,0.037648674100637436,0.7309648990631104,-0.6104761362075806,-0.4671950340270996,-0.1774287223815918,0.07658211886882782,0.14055687189102173,0.8208773136138916,-0.32766446471214294,-2.4929068088531494,0.2463977336883545,0.04188544675707817,0.13054968416690826,-0.04901186004281044,-0.55328369140625,0.5796338319778442,-0.02509256638586521,0.40512844920158386,-0.08190775662660599,-0.2567645013332367,-1.0286487340927124,0.04726196452975273,-0.004778604488819838,0.05393558368086815,-0.6710269451141357,-0.7943394184112549,0.01564222015440464,-0.0031570466235280037,0.44469448924064636,-0.12206459045410156,0.0883607417345047,0.17191773653030396,-0.49501654505729675,-1.1419334411621094,-0.6015967726707458,0.11936161667108536,0.16284212470054626,0.04108075797557831,0.5222424864768982,-0.6745365858078003,0.279323548078537,0.03183424845337868,0.6341655850410461,0.12253176420927048,-0.4210871458053589,0.0010546953417360783,-0.074709951877594,0.622814416885376,-0.12238644808530807,0.06882643699645996,0.13811278343200684,-0.4327802360057831,0.7375333309173584,0.016564302146434784,1.594714641571045,0.019644105806946754,-0.10815176367759705,0.3669013977050781,0.018664924427866936,-0.02360021322965622,0.07644889503717422,0.44237640500068665,0.09087049961090088,-0.12302176654338837,0.0884336307644844,-0.14625121653079987,-0.19165758788585663,0.41762587428092957,-0.9777200222015381,0.04788313806056976,0.033588409423828125,0.34871813654899597,0.15808366239070892,0.019515346735715866,-0.023146092891693115,0.06893431395292282,0.2285512387752533,-0.18341027200222015,-0.5751174688339233,-0.36939898133277893,-0.03275574743747711,0.4730638265609741,-0.16342777013778687,-0.15220026671886444,-0.17292356491088867,-0.10343286395072937,-0.0521371029317379,0.10805824398994446,0.7526788115501404,0.024565888568758965,0.4659041166305542,-0.44844678044319153,0.02103183977305889,-0.6692301034927368,-0.9086192846298218,-0.3329666256904602,-0.011517814360558987,0.08203161507844925,-0.026022404432296753,0.020852021872997284,0.05847150459885597,-0.3380585014820099,0.02375953085720539,-0.19595952332019806,-0.317535400390625,-0.009173045866191387,0.5851259827613831,-0.11623859405517578,-0.3031575679779053,0.16720503568649292,0.8505573272705078,-0.5125932693481445,0.04516049847006798,-0.31609758734703064,-0.1878652423620224,0.11536440253257751,-0.22212009131908417,0.23099388182163239,-0.614861249923706,-0.28029665350914,0.09743794798851013,-0.5797655582427979,0.0662219449877739,0.7701377272605896,0.6347107887268066,0.32570600509643555,0.11268153786659241,0.5146111845970154,-0.07292747497558594,-0.4935387670993805,0.02157655917108059,0.2546398341655731,0.4850271940231323,0.2640671730041504,-0.024320000782608986,0.2829391360282898,-0.8567065596580505,-0.4108476936817169,0.436583012342453,-0.09978970885276794,-0.5173074007034302,0.8575656414031982,0.005230854731053114,-0.1279856562614441,-0.3146808445453644,-0.7005557417869568,-0.14306806027889252,-0.14926114678382874,0.03200983628630638,0.138682559132576,0.41315698623657227,0.9027156829833984,0.11234454810619354,-0.06427805125713348,-0.44126904010772705,0.06869416683912277,0.8011420965194702,0.1367422342300415,-0.2453097701072693,-0.4157976806163788,0.6615568399429321,0.3775615394115448,0.09212712943553925,-0.5280764102935791,0.08719594776630402,-0.5754348635673523,-0.17867650091648102,-0.013844221830368042,-0.8060533404350281,0.3489616811275482,0.043812498450279236,0.10807382315397263,-0.450777530670166,0.050905726850032806,-0.5945717096328735,-0.2845263183116913,-0.605440616607666,-0.12278025597333908,-0.41136425733566284,-0.2581045627593994,-0.6694736480712891,0.8712796568870544,-0.145674929022789,-0.1684076189994812,-0.782008171081543,-0.635816752910614,0.8695326447486877,0.44408562779426575,0.38650843501091003,0.21173904836177826,-0.4705639183521271,0.2550830841064453,0.2923412024974823,0.3512745201587677,0.22907868027687073,-0.7130409479141235,0.5509024262428284,0.010968644171953201,-0.005012353882193565,-0.39720040559768677,-0.7016420960426331,0.12888549268245697,-0.39099276065826416,-0.6822554469108582,0.0331951268017292,0.2453330010175705,-0.22294315695762634,-0.1169835552573204,-0.026468463242053986,-0.20593643188476562,0.24373413622379303,0.032548483461141586,-1.2902132272720337,0.32080748677253723,-0.4612255394458771,-0.07766743004322052,-0.02791076898574829,-0.23326848447322845,-0.6128444671630859,0.1824282705783844,0.048099637031555176,-0.05291151627898216,-0.4834796190261841,-0.5297260284423828,-0.3019828498363495,-0.2398214340209961,-0.6461482048034668,1.4191383123397827,-0.03955334424972534,-0.20886215567588806,-0.079178087413311,0.269137442111969,0.003847976913675666,-0.17100223898887634,0.27691197395324707,-0.0499231219291687,-0.05549802631139755,0.46382033824920654,-0.14077919721603394,0.17093682289123535,-0.40779030323028564,0.13826106488704681,-0.019555432721972466,-0.9126490950584412,0.3210179805755615,0.15113955736160278,-0.7253900766372681,0.04781994968652725,-0.5756023526191711,0.28289464116096497,-0.7556679248809814,0.012746543623507023,-0.44452381134033203,0.31316909193992615,0.6388196349143982,0.044480081647634506,0.055366430431604385,-0.14206713438034058,0.2775479257106781,-1.0222281217575073,0.04189753532409668,-0.37061020731925964,0.093056321144104,-0.6548654437065125,0.1008307933807373,0.5301709175109863,0.22183988988399506,0.23505257070064545,-0.07387255877256393,0.611284613609314,0.4458273649215698,-0.010061605833470821,-0.25031834840774536,-0.212351456284523,-1.1173150539398193,-0.4063624441623688,0.22727899253368378,0.2440338283777237,0.5671586394309998,0.13776183128356934,0.022582046687602997,0.4141749143600464,-0.0006022097659297287,0.12165260314941406,0.00017807335825636983,-0.012964322231709957,0.049511853605508804,-0.49065926671028137,-0.4583936929702759,0.2155439704656601,-0.7611148953437805,0.1469474881887436,-0.03765787184238434,0.4129224121570587,0.10351614654064178,0.05257958173751831,-0.9028699994087219,0.09684872627258301,-0.07256414741277695,-0.02889292873442173,0.08338988572359085,-0.24617479741573334,0.5423755645751953,-0.13542711734771729,-0.5938572883605957,-0.15647576749324799,0.376882940530777,-0.270038902759552,-0.3860075771808624,-0.07925038039684296,-0.027232609689235687,-0.08387935161590576,0.06057542935013771,-0.7801065444946289,-0.4185299873352051,-0.7922973036766052,0.21627944707870483,0.21349883079528809,-0.5039819478988647,-0.48802250623703003,0.08444855362176895,0.7090664505958557,0.26078060269355774,-0.0792367085814476,-0.015847600996494293,0.03219491243362427,-0.09739115089178085,0.023694908246397972,0.5398148894309998,0.2530229091644287,0.2428293377161026,-0.399978369474411,-0.3268192708492279,-0.04889136552810669,0.048063699156045914,0.0066806417889893055,0.08003591746091843,1.493631862103939e-05,-0.36576956510543823,-0.07758568972349167,-0.19925378262996674,0.32828542590141296,-0.9263343214988708,0.26728320121765137,0.5554590821266174,-0.0551457554101944,0.4875740706920624,-0.12086685001850128,0.0025136065669357777,-0.5472445487976074,0.15668132901191711,0.6041079163551331,-0.10946837067604065,0.7633246779441833,-0.20734018087387085,0.26546069979667664,-0.8269662857055664,-0.6963504552841187,-0.19915010035037994,0.22136163711547852,-0.06196295842528343,-0.743241548538208,-0.08954954892396927,-0.16692648828029633,-0.12000754475593567,-0.03186648711562157,-0.11939430236816406,0.19033688306808472,-0.325954407453537,-0.25996115803718567,-0.055988430976867676,0.11344555020332336,-0.030988330021500587,-0.008147685788571835,0.053445003926754,0.02446887642145157,0.06107044219970703,0.05037901550531387,-0.003565093968063593,-0.5549531579017639,0.06581754237413406,0.12493110448122025,0.11079368740320206,-0.057214971631765366,0.38680437207221985,0.13032804429531097,0.9521340727806091,0.0374118871986866,-0.20769795775413513,-0.201871857047081,-0.09935863316059113,-0.11061464995145798,-0.4870190918445587,-0.6163665652275085,0.18073982000350952,-0.1383209079504013,0.45063352584838867,0.1868775486946106,0.008494099602103233,-0.11836979538202286,-0.20479227602481842,0.7685545086860657,-0.06125699356198311,0.1153419241309166,0.12428271025419235,0.4081973731517792,-0.7014006972312927,0.8397367000579834,-0.7303571701049805,-0.642005980014801,0.02683025598526001,-0.1390785127878189,0.5474317073822021,0.02303793467581272,-0.18314526975154877,0.08464766293764114,-0.6925125122070312,-0.7089852690696716,-0.006122198887169361,-0.3562637269496918,-0.32754820585250854,1.0329264402389526,-0.4582597613334656,0.006977106910198927,0.5993576645851135,-0.16391034424304962,0.04758717864751816,0.6612189412117004,0.42824599146842957,-0.09782406687736511,0.04339921846985817,-0.20552708208560944,0.01766418106853962,0.815248429775238,1.0901768207550049,0.30321204662323,0.9702644944190979,-0.307131290435791,0.26524806022644043,0.347001850605011,0.12864932417869568,0.3884672224521637,0.7333967685699463,0.31533390283584595,-0.47204434871673584,-0.06338153779506683,0.23463593423366547,-0.09875573962926865,-1.2841070890426636,0.8276045322418213,0.523890495300293,-0.6525658965110779,-0.0996217280626297,0.23559938371181488,0.04355474188923836,0.8361559510231018,0.21281199157238007,0.050779785960912704,0.07482172548770905,0.3883603811264038,0.43972742557525635,0.8688217401504517,0.560851514339447,-0.25913283228874207,0.33996251225471497,0.34257519245147705,0.17805761098861694,0.1878308355808258,0.02960076741874218,-0.0735221728682518,0.9327507615089417,-0.2038869708776474,-0.18546845018863678,0.1039937436580658,-0.09277608245611191,0.14202947914600372,0.07053404301404953,0.5229926109313965,0.2132868617773056,0.035386133939027786,0.6741551756858826,0.24254725873470306,0.44525769352912903,0.12397582083940506,-0.2863849997520447,0.559693455696106,0.14686626195907593,0.29539892077445984,-0.12650278210639954,0.06291842460632324,-0.8422624468803406,0.04237041994929314,0.007389382924884558,0.37463176250457764,0.10682480782270432,-0.03198730945587158,0.465615451335907,0.16351723670959473,0.23626090586185455,0.5743640661239624,-0.06770531833171844,-0.36576318740844727,-0.025006933137774467,0.6642703413963318,-0.3570851981639862,-0.6556174755096436,-0.06077509745955467,-0.3137376606464386,0.3435794711112976,-0.28338661789894104,-0.5795745849609375,0.28138694167137146,-0.0940452516078949,-0.39014387130737305,-0.040398579090833664,-0.5216756463050842,0.08973458409309387,0.07243391871452332,0.050507571548223495,-0.650305986404419,-0.08357929438352585,-0.3129682242870331,-0.16673369705677032,0.03893517330288887,-0.3680467903614044,-0.5348758697509766,-0.5680378079414368,0.13313661515712738,-0.39294955134391785,0.22156482934951782,0.3337111473083496,-0.1362757831811905,0.08571089804172516,0.9414482712745667,0.2162836194038391,0.6077156662940979,0.1500379741191864,0.9411706328392029,0.008443251252174377,-0.6186343431472778,0.5936334133148193,-0.28293606638908386,0.7178182601928711,-0.197199285030365,-0.4087033271789551,-0.15369558334350586,0.24408601224422455,-0.033993300050497055,0.4028724431991577,-0.47142884135246277,0.365502268075943,-0.7970565557479858,-0.2018960565328598,0.6035148501396179,-0.020830778405070305,-0.019620271399617195,-0.3390999734401703,0.14127680659294128,-0.036767102777957916,-0.08718038350343704,0.051093295216560364,-0.7027124166488647,0.3727489411830902,0.696155846118927,0.34525051712989807,-0.28541135787963867,0.5212035179138184,-0.3950139880180359,0.31093549728393555,-1.0923060178756714,-0.1637863963842392,-0.6693952083587646,-0.7633885741233826,-0.08204400539398193,0.10380899906158447,-0.49718353152275085,-0.9957800507545471,0.24063564836978912,0.3240232467651367,-0.29527801275253296,-0.3443204164505005,0.33433008193969727,0.03903636708855629,-0.31652840971946716,-0.6079978346824646,-0.16567549109458923,-0.0010877038585022092,-0.3227839469909668,0.14381907880306244,0.11739552766084671,-0.27012357115745544,-0.024931972846388817,0.009826506488025188,-0.0010996657656505704,0.20802055299282074,-0.3982342779636383,-0.06783831864595413])
eb = torch.DoubleTensor([-0.04483657702803612,0.04773061349987984,0.37317752838134766,-0.186677947640419,0.6871497631072998,0.17067314684391022,0.501641571521759,0.2133561372756958,-0.2533939778804779,-0.2607075870037079,-0.08918702602386475,0.02896830253303051,-0.6273435354232788,0.37112268805503845,-0.33218398690223694,0.8587245345115662,1.0187979936599731,-0.023201018571853638,-0.29323357343673706,-0.22788546979427338,0.21896551549434662,-0.03456881642341614,0.3109235465526581,0.7113319635391235,0.5663745999336243,0.05559079721570015,0.5355936288833618,0.03617621213197708,-0.21763235330581665,0.2401406317949295,-0.03502405434846878,-0.00023508661251980811,0.2147354930639267,-0.2282571792602539,-0.34072285890579224,0.2235729992389679,-0.4243127107620239,-0.2091415524482727,-0.6481261849403381,-0.05592655763030052,-0.8375883102416992,-0.5731526017189026,-0.49358272552490234,0.19234324991703033,0.23039765655994415,0.13207022845745087,-0.31395867466926575,0.7963513135910034,-0.15963080525398254,-0.09391775727272034,-0.6372794508934021,0.03966215252876282,0.18924498558044434,0.09397304058074951,0.20545470714569092,0.42477330565452576,-0.15290428698062897,-0.6923989057540894,-0.46341845393180847,-0.3296276330947876,-0.058417171239852905,0.4835377335548401,0.5274681448936462,-1.028611183166504,0.13369297981262207,-0.03468633070588112,0.22843004763126373,0.7035647034645081,-1.1278095245361328,-0.8125713467597961,-0.1658022105693817,-0.04798814654350281,-0.3586948812007904,-0.4885730743408203,0.12819407880306244,0.12464934587478638,-0.39259472489356995,0.7512699365615845,0.35312002897262573,0.0754275992512703,-0.377669095993042,-0.013752118684351444,0.12385492026805878,0.7057179808616638,-0.0001628780155442655,-0.330280601978302,-0.11212083697319031,0.14383076131343842,0.008898808620870113,0.4181112051010132,-0.8438806533813477,-0.7327632308006287,0.1091092973947525,0.16112357378005981,0.4458189308643341,0.01071947067975998,0.5114210844039917,-0.0700284093618393,-0.18531781435012817,-0.00619296170771122,0.07539021223783493,-0.4975631833076477,0.27431535720825195,0.014520647004246712,-0.2660548686981201,-0.1840011179447174,-0.07901358604431152,-0.8249804973602295,-0.030823802575469017,0.6500431299209595,-0.056753113865852356,-0.22013328969478607,-0.1663818508386612,-0.6812082529067993,-0.5514544248580933,0.06828171014785767,0.19242407381534576,-0.6200861930847168,-0.007494610734283924,0.012174193747341633,0.6541118621826172,-0.030345093458890915,0.10187788307666779,1.399490475654602,-0.16060003638267517,-0.13424493372440338,-0.18341469764709473,0.7047732472419739,-1.0100269317626953,-0.1528884470462799,0.24312308430671692,0.4140414297580719,0.022335482761263847,0.0038727542851120234,-0.45979392528533936,-0.5795531868934631,0.38564634323120117,-0.07588528841733932,-0.21666915714740753,0.23805351555347443,0.12166573107242584,-0.3613172471523285,0.02576027810573578,0.19949860870838165,0.0984051451086998,0.049369148910045624,-0.025439439341425896,-0.6912478804588318,-0.06677880883216858,-0.08513939380645752,-0.0027560556773096323,-0.05050461366772652,-0.1790103167295456,0.023896142840385437,-0.26300960779190063,0.10361181944608688,-0.04462920501828194,0.33661791682243347,-0.07449043542146683,0.0335581935942173,0.4273805320262909,-0.029812538996338844,0.3094140589237213,-0.2700003981590271,0.5862663984298706,0.13043245673179626,-0.015343000181019306,0.9306020140647888,-0.1774941235780716,0.4560281038284302,-0.08511859178543091,0.03805619850754738,0.7308606505393982,-0.6001361012458801,-0.4657292664051056,-0.1770072877407074,0.07713034749031067,0.1403573453426361,0.8212491273880005,-0.3264608681201935,-2.515613317489624,0.24581247568130493,0.042459677904844284,0.13089537620544434,-0.04899904504418373,-0.5541562438011169,0.5794254541397095,-0.024606527760624886,0.4056221544742584,-0.08129971474409103,-0.2627589702606201,-1.0254366397857666,0.04728496074676514,0.0026430864818394184,0.05330421030521393,-0.6699351668357849,-0.7858867049217224,0.015646830201148987,-0.0029799267649650574,0.44962918758392334,-0.120989590883255,0.08811937272548676,0.1726214736700058,-0.4947623014450073,-1.1427956819534302,-0.6029757857322693,0.11947871744632721,0.16291476786136627,0.04109657183289528,0.5325021147727966,-0.6638621091842651,0.2791779339313507,0.031778596341609955,0.6343069076538086,0.12191664427518845,-0.4195927381515503,0.0003955585416406393,-0.07450604438781738,0.6218315958976746,-0.12029573321342468,0.0691620334982872,0.13674944639205933,-0.4339338541030884,0.738448977470398,0.016290685161948204,1.588141679763794,0.020070675760507584,-0.10806283354759216,0.36840638518333435,0.01966589130461216,-0.02352425828576088,0.07661353796720505,0.44595277309417725,0.09122729301452637,-0.12282276153564453,0.08840099722146988,-0.14639733731746674,-0.17947140336036682,0.413542777299881,-0.9755143523216248,0.04788714647293091,0.03393468260765076,0.3501087725162506,0.15669623017311096,0.01989574544131756,-0.022869879379868507,0.06806905567646027,0.22855953872203827,-0.18383662402629852,-0.5737954378128052,-0.36332452297210693,-0.03308074176311493,0.4734649658203125,-0.15984612703323364,-0.1523541659116745,-0.17264500260353088,-0.1032862737774849,-0.05216356739401817,0.1081317588686943,0.7478225231170654,0.020197110250592232,0.4658114016056061,-0.4478786289691925,0.013502907939255238,-0.6695755124092102,-0.9025859832763672,-0.33196473121643066,-0.01214214414358139,0.08192849159240723,-0.027328966185450554,0.02130632847547531,0.05845024436712265,-0.3333896994590759,0.018082205206155777,-0.19586588442325592,-0.3160017132759094,-0.00901811569929123,0.5860122442245483,-0.1203141063451767,-0.30158188939094543,0.1672697365283966,0.836729109287262,-0.49386030435562134,0.04441872239112854,-0.3157680928707123,-0.18801595270633698,0.11524596810340881,-0.22183352708816528,0.23050546646118164,-0.614525556564331,-0.2800673842430115,0.09704499691724777,-0.5807593464851379,0.06685572862625122,0.7688570618629456,0.6355525255203247,0.32607024908065796,0.10387933999300003,0.5157882571220398,-0.07595648616552353,-0.4934520721435547,0.021463898941874504,0.2539072632789612,0.4839949905872345,0.26433998346328735,-0.024168262258172035,0.2907170355319977,-0.850746214389801,-0.4487314224243164,0.436367005109787,-0.0999445915222168,-0.5157395005226135,0.8563234806060791,0.005692305509001017,-0.12822531163692474,-0.3138269782066345,-0.6999340653419495,-0.142827108502388,-0.1488122195005417,0.031788893043994904,0.13871443271636963,0.4135340452194214,0.9034778475761414,0.11223997920751572,-0.06389141827821732,-0.4409245252609253,0.06887771934270859,0.7965441346168518,0.14496326446533203,-0.2466164231300354,-0.416379451751709,0.6613852381706238,0.3687567710876465,0.09309730678796768,-0.51253741979599,0.08173082768917084,-0.5578972697257996,-0.16846910119056702,-0.013495334424078465,-0.8067273497581482,0.3492184281349182,0.044502127915620804,0.10048496723175049,-0.4511299431324005,0.05171045660972595,-0.594784677028656,-0.2855786979198456,-0.6056913137435913,-0.122214175760746,-0.4121432304382324,-0.25599074363708496,-0.668307363986969,0.8702410459518433,-0.14592453837394714,-0.16806259751319885,-0.780672013759613,-0.6366919875144958,0.8569228649139404,0.44420307874679565,0.3867206573486328,0.21137139201164246,-0.47142982482910156,0.25530651211738586,0.29330968856811523,0.3354114294052124,0.2284901738166809,-0.7210278511047363,0.5487932562828064,0.010792954824864864,-0.004367903806269169,-0.3959106206893921,-0.690689742565155,0.12801384925842285,-0.3819425702095032,-0.68174809217453,0.03268415480852127,0.23972362279891968,-0.22058439254760742,-0.11632361263036728,-0.025601770728826523,-0.20012813806533813,0.24413201212882996,0.03250044584274292,-1.2867764234542847,0.3258330225944519,-0.4609893560409546,-0.0777033343911171,-0.028459792956709862,-0.23187977075576782,-0.6127241849899292,0.17894981801509857,0.047546859830617905,-0.05281808227300644,-0.47832635045051575,-0.5297309160232544,-0.30029529333114624,-0.2394408881664276,-0.6453726291656494,1.4215391874313354,-0.039078764617443085,-0.20831793546676636,-0.07982137799263,0.2601466178894043,0.0039029340259730816,-0.17138956487178802,0.2758135497570038,-0.049242857843637466,-0.056081946939229965,0.4616350829601288,-0.14042843878269196,0.1705615520477295,-0.40644344687461853,0.13761702179908752,-0.01971108466386795,-0.9113693833351135,0.31172993779182434,0.150885671377182,-0.7229619026184082,0.04740600660443306,-0.5749373435974121,0.28253504633903503,-0.7537394165992737,0.012357830069959164,-0.43729448318481445,0.3128240704536438,0.635767936706543,0.04445880278944969,0.0549982488155365,-0.13820096850395203,0.27776628732681274,-1.0180768966674805,0.04166305437684059,-0.3637579083442688,0.09372901171445847,-0.6555085182189941,0.10085070133209229,0.5217609405517578,0.22240233421325684,0.2358710616827011,-0.0737641304731369,0.6020964980125427,0.44706249237060547,-0.010007830336689949,-0.24931195378303528,-0.2127111554145813,-1.1140040159225464,-0.40626028180122375,0.22742728888988495,0.24395208060741425,0.5671634674072266,0.13764803111553192,0.022432995960116386,0.41377687454223633,-0.0010470300912857056,0.12099288403987885,0.0002855106722563505,-0.013398323208093643,0.049157653003931046,-0.4899083375930786,-0.45774146914482117,0.21456190943717957,-0.7610188722610474,0.14690518379211426,-0.03805851936340332,0.4121890366077423,0.10386917740106583,0.05232514813542366,-0.9033927917480469,0.0969940796494484,-0.07218392193317413,-0.029159268364310265,0.08302932977676392,-0.24656184017658234,0.5173870921134949,-0.13549137115478516,-0.5937395691871643,-0.1559753715991974,0.37669843435287476,-0.26996782422065735,-0.3857892155647278,-0.08008790016174316,-0.026905607432127,-0.08320724964141846,0.060744594782590866,-0.7794590592384338,-0.4175705909729004,-0.7918530702590942,0.21591712534427643,0.21367128193378448,-0.5020452737808228,-0.48738670349121094,0.08530799299478531,0.709013044834137,0.2612271308898926,-0.07826327532529831,-0.015890099108219147,0.032193779945373535,-0.09558065980672836,0.024111008271574974,0.5401906371116638,0.25277405977249146,0.23684339225292206,-0.39345964789390564,-0.32586920261383057,-0.04922620579600334,0.048341646790504456,0.006848477758467197,0.08045695722103119,0.00011158658162457868,-0.3647189736366272,-0.07366973161697388,-0.19362905621528625,0.32761651277542114,-0.9340934753417969,0.268552303314209,0.5566644668579102,-0.05492723733186722,0.49181875586509705,-0.12043515592813492,0.002613810822367668,-0.534089207649231,0.1567520946264267,0.5862239003181458,-0.10883393883705139,0.7626762390136719,-0.20681703090667725,0.2656417489051819,-0.8396252989768982,-0.6972710490226746,-0.1988091617822647,0.2216513454914093,-0.06183731555938721,-0.7422757744789124,-0.08900261670351028,-0.1664542853832245,-0.12155577540397644,-0.031593769788742065,-0.11902736127376556,0.18987877666950226,-0.32493922114372253,-0.2599993348121643,-0.05598672106862068,0.11207536607980728,-0.030879925936460495,-0.008186882361769676,0.054692983627319336,0.024065759032964706,0.05988157540559769,0.05040425434708595,-0.0031639798544347286,-0.5545583367347717,0.06577317416667938,0.12476418912410736,0.11132144927978516,-0.05683034658432007,0.38610491156578064,0.12996676564216614,0.9536406397819519,0.03672194853425026,-0.20745201408863068,-0.20214153826236725,-0.09987735748291016,-0.11019005626440048,-0.47852644324302673,-0.6152728796005249,0.1800217479467392,-0.13811570405960083,0.4509047269821167,0.18732498586177826,0.008120580576360226,-0.11769925802946091,-0.20738106966018677,0.7595991492271423,-0.0634431317448616,0.1151202917098999,0.1246606633067131,0.4076233208179474,-0.707216203212738,0.8321130275726318,-0.7288554310798645,-0.6422615647315979,0.02631169557571411,-0.13831840455532074,0.5463858246803284,0.022911200299859047,-0.16704051196575165,0.08468011766672134,-0.6916055083274841,-0.7130104303359985,-0.006149894092231989,-0.3553719222545624,-0.3243018388748169,1.0373620986938477,-0.4457740783691406,0.00714068440720439,0.6022213101387024,-0.16344623267650604,0.047031763941049576,0.6662301421165466,0.4240074157714844,-0.09762250632047653,0.043754130601882935,-0.18981409072875977,0.018277961760759354,0.8168532848358154,1.0886380672454834,0.30279305577278137,0.9728781580924988,-0.3067699074745178,0.2667279839515686,0.3467195928096771,0.1286776065826416,0.38936710357666016,0.7332115173339844,0.3159586489200592,-0.4687553346157074,-0.06365004926919937,0.23497679829597473,-0.10023687779903412,-1.2856099605560303,0.8281229138374329,0.508371889591217,-0.6381419897079468,-0.10013460367918015,0.23522555828094482,0.04346565529704094,0.8401005268096924,0.21160127222537994,0.042429644614458084,0.07475772500038147,0.3793920874595642,0.4390912353992462,0.8694427609443665,0.561264157295227,-0.25919821858406067,0.3402272164821625,0.3430721163749695,0.1761987954378128,0.18798492848873138,0.030269606038928032,-0.07326804846525192,0.9180602431297302,-0.20389367640018463,-0.18576215207576752,0.10432517528533936,-0.09204495698213577,0.14236336946487427,0.07045645266771317,0.5229149460792542,0.20633608102798462,0.03538264334201813,0.674169659614563,0.24297688901424408,0.44288918375968933,0.12409254908561707,-0.2864972949028015,0.5599389672279358,0.1468644142150879,0.2999482750892639,-0.12674489617347717,0.06240829825401306,-0.8430826663970947,0.04253102466464043,0.007329321000725031,0.37518173456192017,0.10641989856958389,-0.03171148896217346,0.46684888005256653,0.16349317133426666,0.23627418279647827,0.5699340105056763,-0.06780961900949478,-0.3658393323421478,-0.025172220543026924,0.6637865900993347,-0.3456341028213501,-0.6542292833328247,-0.0606868751347065,-0.3135889172554016,0.34403184056282043,-0.2831581234931946,-0.5782094597816467,0.2810118496417999,-0.09422948956489563,-0.37822815775871277,-0.040243539959192276,-0.5206443071365356,0.08989804238080978,0.07309509813785553,0.05050193518400192,-0.6477607488632202,-0.0829831063747406,-0.3137882351875305,-0.16598215699195862,0.039729077368974686,-0.36857274174690247,-0.5324832201004028,-0.5757255554199219,0.13266944885253906,-0.3936109244823456,0.22142356634140015,0.33314135670661926,-0.1364988088607788,0.08461729437112808,0.9400883316993713,0.21602106094360352,0.5909544229507446,0.15111063420772552,0.9414671659469604,0.008623681962490082,-0.6108160018920898,0.5922940969467163,-0.2869167625904083,0.7256043553352356,-0.19063082337379456,-0.40712639689445496,-0.1532309353351593,0.2373686134815216,-0.03265313804149628,0.4018058776855469,-0.4654693305492401,0.3659631609916687,-0.8080635070800781,-0.18889762461185455,0.604720413684845,-0.02130243554711342,-0.020534394308924675,-0.338440865278244,0.14068055152893066,-0.03700077161192894,-0.08719855546951294,0.05103693902492523,-0.7027367949485779,0.37223193049430847,0.6886561512947083,0.34404265880584717,-0.2847321331501007,0.5211305022239685,-0.3949347138404846,0.3100195527076721,-1.0910130739212036,-0.16386272013187408,-0.6807584166526794,-0.7441275119781494,-0.08232355117797852,0.10300814360380173,-0.4969278872013092,-0.99323970079422,0.2407221794128418,0.32448628544807434,-0.2957044839859009,-0.34380051493644714,0.3356289565563202,0.03855271264910698,-0.30912113189697266,-0.6067081689834595,-0.1545432060956955,-0.00134187750518322,-0.3222282826900482,0.14313960075378418,0.117787204682827,-0.2750161290168762,-0.025468328967690468,0.0098362872377038,-0.00011920707038370892,0.20909541845321655,-0.3985205888748169,-0.06723182648420334])

In [15]:
ea.shape

torch.Size([768])

In [16]:
ea.mean()

tensor(-0.0163, dtype=torch.float64)

In [17]:
eb.mean()

tensor(-0.0162, dtype=torch.float64)

In [18]:
import gzip
import io

In [19]:
def compress(embed):
    bout = io.BytesIO()
    with gzip.GzipFile(fileobj=bout, mode="w") as fout:
        np.save(fout, embed.double().detach().numpy().astype(np.float64))
    return bout.getvalue()

In [21]:
len(compress(ea))

3655

In [22]:
len(compress(eb))

3676

In [28]:
ea.shape[0] * ea.element_size(), ea.element_size()

(6144, 8)

In [31]:
def decompress(content: bytes) -> torch.Tensor:
    binp = io.BytesIO(content)
    with gzip.GzipFile(fileobj=binp, mode="r") as finp:
        return torch.DoubleTensor(np.load(finp))

In [33]:
(ea == decompress(compress(ea))).all()

tensor(True)

In [34]:
(eb == decompress(compress(eb))).all()

tensor(True)